In [1]:
import os
import requests
import rasterio
from rasterio.mask import mask
from shapely.geometry import box
from io import BytesIO

# Define the URL of the GeoTIFF files
tif_files = ['https://storage.googleapis.com/geo-assets/C3S-LC-L4-LCCS-Map-300m-P1Y/ESACCI-LC-L4-LCCS-Map-300m-P1Y-1992-v2.0.7_cog.tif',
'https://storage.googleapis.com/geo-assets/C3S-LC-L4-LCCS-Map-300m-P1Y/ESACCI-LC-L4-LCCS-Map-300m-P1Y-1993-v2.0.7_cog.tif',
'https://storage.googleapis.com/geo-assets/C3S-LC-L4-LCCS-Map-300m-P1Y/ESACCI-LC-L4-LCCS-Map-300m-P1Y-1994-v2.0.7_cog.tif',
'https://storage.googleapis.com/geo-assets/C3S-LC-L4-LCCS-Map-300m-P1Y/ESACCI-LC-L4-LCCS-Map-300m-P1Y-1995-v2.0.7_cog.tif',
'https://storage.googleapis.com/geo-assets/C3S-LC-L4-LCCS-Map-300m-P1Y/ESACCI-LC-L4-LCCS-Map-300m-P1Y-1996-v2.0.7_cog.tif',
'https://storage.googleapis.com/geo-assets/C3S-LC-L4-LCCS-Map-300m-P1Y/ESACCI-LC-L4-LCCS-Map-300m-P1Y-1997-v2.0.7_cog.tif',
'https://storage.googleapis.com/geo-assets/C3S-LC-L4-LCCS-Map-300m-P1Y/ESACCI-LC-L4-LCCS-Map-300m-P1Y-1998-v2.0.7_cog.tif',
'https://storage.googleapis.com/geo-assets/C3S-LC-L4-LCCS-Map-300m-P1Y/ESACCI-LC-L4-LCCS-Map-300m-P1Y-1999-v2.0.7_cog.tif',
'https://storage.googleapis.com/geo-assets/C3S-LC-L4-LCCS-Map-300m-P1Y/ESACCI-LC-L4-LCCS-Map-300m-P1Y-2000-v2.0.7_cog.tif',
'https://storage.googleapis.com/geo-assets/C3S-LC-L4-LCCS-Map-300m-P1Y/ESACCI-LC-L4-LCCS-Map-300m-P1Y-2001-v2.0.7_cog.tif',
'https://storage.googleapis.com/geo-assets/C3S-LC-L4-LCCS-Map-300m-P1Y/ESACCI-LC-L4-LCCS-Map-300m-P1Y-2002-v2.0.7_cog.tif',
'https://storage.googleapis.com/geo-assets/C3S-LC-L4-LCCS-Map-300m-P1Y/ESACCI-LC-L4-LCCS-Map-300m-P1Y-2003-v2.0.7_cog.tif',
'https://storage.googleapis.com/geo-assets/C3S-LC-L4-LCCS-Map-300m-P1Y/ESACCI-LC-L4-LCCS-Map-300m-P1Y-2004-v2.0.7_cog.tif',
'https://storage.googleapis.com/geo-assets/C3S-LC-L4-LCCS-Map-300m-P1Y/ESACCI-LC-L4-LCCS-Map-300m-P1Y-2005-v2.0.7_cog.tif',
'https://storage.googleapis.com/geo-assets/C3S-LC-L4-LCCS-Map-300m-P1Y/ESACCI-LC-L4-LCCS-Map-300m-P1Y-2006-v2.0.7_cog.tif',
'https://storage.googleapis.com/geo-assets/C3S-LC-L4-LCCS-Map-300m-P1Y/ESACCI-LC-L4-LCCS-Map-300m-P1Y-2007-v2.0.7_cog.tif',
'https://storage.googleapis.com/geo-assets/C3S-LC-L4-LCCS-Map-300m-P1Y/ESACCI-LC-L4-LCCS-Map-300m-P1Y-2008-v2.0.7_cog.tif',
'https://storage.googleapis.com/geo-assets/C3S-LC-L4-LCCS-Map-300m-P1Y/ESACCI-LC-L4-LCCS-Map-300m-P1Y-2009-v2.0.7_cog.tif',
'https://storage.googleapis.com/geo-assets/C3S-LC-L4-LCCS-Map-300m-P1Y/ESACCI-LC-L4-LCCS-Map-300m-P1Y-2010-v2.0.7_cog.tif',
'https://storage.googleapis.com/geo-assets/C3S-LC-L4-LCCS-Map-300m-P1Y/ESACCI-LC-L4-LCCS-Map-300m-P1Y-2011-v2.0.7_cog.tif',
'https://storage.googleapis.com/geo-assets/C3S-LC-L4-LCCS-Map-300m-P1Y/ESACCI-LC-L4-LCCS-Map-300m-P1Y-2012-v2.0.7_cog.tif',
'https://storage.googleapis.com/geo-assets/C3S-LC-L4-LCCS-Map-300m-P1Y/ESACCI-LC-L4-LCCS-Map-300m-P1Y-2013-v2.0.7_cog.tif',
'https://storage.googleapis.com/geo-assets/C3S-LC-L4-LCCS-Map-300m-P1Y/ESACCI-LC-L4-LCCS-Map-300m-P1Y-2014-v2.0.7_cog.tif',
'https://storage.googleapis.com/geo-assets/C3S-LC-L4-LCCS-Map-300m-P1Y/ESACCI-LC-L4-LCCS-Map-300m-P1Y-2015-v2.0.7_cog.tif',
'https://storage.googleapis.com/geo-assets/C3S-LC-L4-LCCS-Map-300m-P1Y/C3S-LC-L4-LCCS-Map-300m-P1Y-2016-v2.1.1_cog.tif',
'https://storage.googleapis.com/geo-assets/C3S-LC-L4-LCCS-Map-300m-P1Y/C3S-LC-L4-LCCS-Map-300m-P1Y-2017-v2.1.1_cog.tif',
'https://storage.googleapis.com/geo-assets/C3S-LC-L4-LCCS-Map-300m-P1Y/C3S-LC-L4-LCCS-Map-300m-P1Y-2018-v2.1.1_cog.tif',
'https://storage.googleapis.com/geo-assets/C3S-LC-L4-LCCS-Map-300m-P1Y/C3S-LC-L4-LCCS-Map-300m-P1Y-2019-v2.1.1_cog.tif'

]
# Define the bounding box coordinates
minx, miny, maxx, maxy = 21.8375, 57.5246, 28.1945, 59.6750

# Create a shapely geometry using the bounding box coordinates
bbox = box(minx, miny, maxx, maxy)

# Loop through each GeoTIFF file
for tif_url in tif_files:
    # Download the GeoTIFF file from the URL
    response = requests.get(tif_url)
    tif_content = BytesIO(response.content)

    # Open the GeoTIFF file from the downloaded content
    with rasterio.open(tif_content) as src:
        # Clip the raster to the bounding box geometry
        clipped_data, clipped_transform = mask(dataset=src, shapes=[bbox], crop=True)
        # Update the metadata (e.g., transform, width, height)
        clipped_meta = src.meta.copy()
        clipped_meta.update({"driver": "GTiff",
                             "height": clipped_data.shape[1],
                             "width": clipped_data.shape[2],
                             "transform": clipped_transform})

        # Extract the filename without extension
        filename = os.path.splitext(os.path.basename(tif_url))[0]
        # Define the output filename
        output_filename = f'{filename}_clipped.tif'

        # Write the clipped data to a new GeoTIFF file
        with rasterio.open(output_filename, 'w', **clipped_meta) as dst:
            dst.write(clipped_data)


In [1]:
import os
from dggrid4py import DGGRIDv7
import geopandas
import shapely
